### 登入

In [ ]:
import os
import requests

BASE_URL = 'http://localhost:1234'

def write_token(token):
    if not os.path.exists('token.txt'):
        with open('token.txt', 'w') as f:
            f.write(token)
    else:
        with open('token.txt', 'w') as f:
            f.write(token)

def login(user_name, pwd):
    try:
        response = requests.post(
            f'{BASE_URL}/users/login',
            json={
                'user_name': user_name,
                'pwd': pwd,
            }
        )
        response.raise_for_status()
        token = response.json().get('token', '')
        if token:
            write_token(token)
    except requests.exceptions.RequestException as err:
        print(err)
login(user_name='', pwd='a123456+')

## Def

In [174]:
import json
import time
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

BASE_URL = 'http://localhost:1234'

def read_token(file_path='./token.txt'):
    try:
        with open(file_path, 'r') as file:
            return file.read().strip()
    except IOError as err:
        print(f'Error reading token from file: {err}')
        return None
    

def get_headers(token):
    return {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }

token = read_token()

def _make_get_request(endpoint, params=None):
    """共用的 HTTP GET 請求處理函數"""
    try:
        response = requests.get(f"{BASE_URL}/{endpoint}", headers=get_headers(token), params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as err:
        print(f"Request failed: {err}")
        return None

def register(user_name, pwd, email):
    try:
        response = requests.post(
            f'{BASE_URL}/users/register',
            json={
                'user_name': user_name,
                'pwd': pwd,
                'email': email,
            }
        )
        response.raise_for_status()
        token = response.json().get('token', '')
        if token:
            write_token(token)
    except requests.exceptions.RequestException as err:
        print(err)
            
def get_all_momentum():
    res = _make_get_request('market/momentum')
    print(res)
    return res.get('data')

def get_momentum_by_range(days):
    res = _make_get_request(f'market/momentum/range/{days}')
    print(res)
    return res.get('data')
    
def get_market_weights():
    res = _make_get_request('market/weights')
    print(res)
    return res.get('data')

def get_stock_prices():
    res = _make_get_request('market/stock/prices')
    print(res)
    return res.get('data')

def get_stock_symbols():
    res = _make_get_request('market/stock/symbols')
    print(res)
    return res.get('data')

def get_market_breadth():
    res = _make_get_request('market/stock/breadth')
    print(res)
    return res.get('data')

def get_stock_winners():
    res = _make_get_request('market/stock/winners')
    print(res)
    return res.get('data')

def get_stock_losers():
    res = _make_get_request('market/stock/losers')
    print(res)
    return res.get('data')

def clone_db_transactions():
    res = _make_get_request('transactions/clone')
    print(res)
    return res

def get_all_transactions():
    res = _make_get_request('transactions')
    print(res)
    return res


def parse_date(date_str):
    date_format = '%Y-%m-%d'
    try:
        date = datetime.strptime(date_str, date_format)
        return date.strftime(date_format)
    except ValueError:
        return None

def record_my_transactions(transactions):
    try:
        for stock_id, transaction_type, quantity, price, transaction_date in transactions:
            data = {
                'stock_id': stock_id,
                'transaction_type': transaction_type,
                'quantity': quantity,
                'price': price,
                'transaction_date': parse_date(transaction_date) or datetime.now().strftime('%Y-%m-%d')
            }
            response = requests.post(f'{BASE_URL}/transactions', headers=get_headers(token), json=data)
            if response.status_code in range(200, 300):
                print(f'Successfully inserted: {data}')
            else:
                print(f'Failed to insert: {data}, Status code: {response.status_code}, Response: {response.text}')
            time.sleep(2)
    except requests.exceptions.RequestException as err:
        print(err)

def get_my_portfolio():
    return _make_get_request('portfolio')

def update_my_portfolio(params):
    try:
        response = requests.put(f'{BASE_URL}/portfolio', headers=get_headers(token), json=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as err:
        print(err)

def get_technews_by_keyword(keyword):
    data =  _make_get_request(f"technews/search?keyword={keyword}")
    return data[::-1]
    
def getAllTechnews (page, size):
	if not isinstance(page, int) or not isinstance(size, int) or page <= 0 or size <= 0:
		print('Valid page & size required')
		return []

	params = {
		'page': page,
		'size': size
	}
	data =  _make_get_request('technews', params)
	return data[::-1]
    
def create_one_company_news (params):
	try:
		response = requests.post(f"{BASE_URL}/company-news", headers=get_headers(token), json=params)
		response.raise_for_status()
		return response.json()
	except requests.exceptions.RequestException as err:
		print(err)
    
def get_all_company_news (page, size):
	if not isinstance(page, int) or not isinstance(size, int) or page <= 0 or size <=0:
		print('Valid page & size required')
		return []

	params = {
		'page': page,
		'size': size
	}
	data =  _make_get_request('company-news', params)
	return data[::-1]
    
def get_all_company_news_by_keyword (keyword, page=1, size=10):
	if not isinstance(page, int) or not isinstance(size, int) or page <= 0 or size <=0:
		print('Valid page & size required')
		return
	return _make_get_request('/company-news/query', {
		'page': page,
		'size': size,
        'keyword': keyword
	})

def get_statement_by_symbol(symbol):
    return _make_get_request(f"statements/{symbol}")
    
def get_user_favorite_news():
    res = _make_get_request('userFavorite/technews')
    print(res)
    return res.get('data')
    
def user_subscribe_news(params):
    try:
        url = f"{BASE_URL}/userFavorite/technews"
        response = requests.post(url, headers=get_headers(token), json=params)
        response.raise_for_status()
        data = response.json()
        return data
    except requests.exceptions.RequestException as err:
        print(err)
    
def print_news(news):
    for item in news:
        company_name = item.get('companyName', '')
        news_id = item.get('id', '--')
        title = item.get('title', '--')
        release_time = item.get('release_time', '--')
        web_url = item.get('web_url', '--')
        print(company_name)
        print(f"{news_id} : {title} {release_time} \n")
        if web_url:
            print(web_url)
        print('----------')
        

def draw_pie_chart(obj):
	labels = obj.keys()
	sizes = obj.values()
	colors = plt.get_cmap('Set2').colors
	explode = [0.1] * len(labels)

	plt.figure(figsize=(5, 5))
	plt.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%', colors=colors, startangle=140)
	plt.title('Portfolio Distribution by Stock')
	plt.show()


def draw_line_chart(x_data, y_data, label, title, xlabel, ylabel, interval=10, figsize=(10, 6), color='#efb441'):
    plt.figure(figsize=figsize)
    plt.plot(x_data, y_data, marker='', linestyle='-', color=color, label=label)

    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=interval))
    plt.gcf().autofmt_xdate()
    plt.grid(True)

    plt.legend()

    plt.tight_layout()
    plt.show()
    
def plot_pe_forward_vs_time(data):
    timestamps = []
    pe_forward_values = []
    
    for entry in data:
        created_at = entry.get('createdAt', '')
        pe_forward = entry.get('PE_Forward', None)
        
        if created_at and pe_forward is not None:
            timestamp = datetime.strptime(created_at, "%Y-%m-%dT%H:%M:%S.%fZ")
            timestamps.append(timestamp)
            
            pe_forward_values.append(float(pe_forward))
    
    draw_line_chart(timestamps, pe_forward_values, label='PE Forward', title='PE Forward vs Time', xlabel='Date', ylabel='PE Forward')
    



#### 註冊

In [ ]:
register(user_name='', pwd='', email='')

In [ ]:
res = get_market_weights()
res

#### 市場廣度指標

In [ ]:
breadth = get_market_breadth()
breadth

### 取得動能指標

In [ ]:
# my_data = get_all_momentum()

my_data = get_momentum_by_range(7)
print(my_data[0])
print(my_data[1])
print(my_data[2])
print(my_data[3])
print(my_data[4])
print(my_data[5])

#### 畫出動能

In [ ]:

import pandas as pd

df = pd.DataFrame(my_data)

df['createdAt'] = pd.to_datetime(df['createdAt'])
draw_line_chart(df['createdAt'], df['volume'], label='Momentum', title='Time Series Momentum', xlabel='Time', ylabel='Volume', interval=1)

### 記錄交易

In [ ]:
transactions = [
	# ('AMZN', 'buy', 2, 186.72, '2025-05-06'),
]

In [ ]:
record_my_transactions(transactions)

### MY Portfolio

In [ ]:
symbols = get_stock_symbols()
symbols

In [ ]:
from decimal import Decimal, ROUND_HALF_UP

my_portfolio = get_my_portfolio()
stock_prices = get_stock_prices()
current_price_map = {item['symbol']: item['price'] for item in stock_prices}

PORTFOLIO_USD = 11000 + 4878

def validate_portfolio_item(item):
    required_keys = ['stock_id', 'quantity', 'average_price']
    if not all(key in item for key in required_keys):
        raise ValueError(f"Invalid portfolio item: {item}")
    try:
        quantity = float(item['quantity'])
        average_price = float(item['average_price'])
        if quantity < 0 or average_price < 0:
            raise ValueError(f"Negative quantity or price in item: {item}")
    except (ValueError, TypeError):
        raise ValueError(f"Invalid data type in item: {item}")
    
def process_portfolio(portfolio_dict, usd_value):
    portfolio_dict['USD'] = usd_value
    return dict(sorted(
        {k: v for k, v in portfolio_dict.items() if v > 0}.items(),
        key=lambda item: item[1],
        reverse=True
    ))

portfolio_values = {}
portfolio_with_current_price = {}

for item in my_portfolio:
    validate_portfolio_item(item)
    stock_id = item['stock_id']
    quantity = Decimal(str(item['quantity']))
    average_price = Decimal(str(item['average_price']))
    current_price = Decimal(str(current_price_map.get(stock_id, item['average_price'])))
    
    # 計算成本價值
    total_cost = (quantity * average_price).quantize(Decimal('0.01'), rounding=ROUND_HALF_UP)
    portfolio_values[stock_id] = portfolio_values.get(stock_id, 0) + float(total_cost)
    
    # 計算當前市場價值
    total_current = (quantity * current_price).quantize(Decimal('0.01'), rounding=ROUND_HALF_UP)
    portfolio_with_current_price[stock_id] = portfolio_with_current_price.get(stock_id, 0) + float(total_current)
    
    print(f"{stock_id}: {float(quantity):>6.2f} * ${float(average_price):>8.2f} = ${float(total_cost):>10.2f}")

# 處理和排序投資組合
sorted_portfolio_values = process_portfolio(portfolio_values, PORTFOLIO_USD)
sorted_current_values = process_portfolio(portfolio_with_current_price, PORTFOLIO_USD)

print('成本：')
draw_pie_chart(sorted_portfolio_values)
print('現在價位：')
draw_pie_chart(sorted_current_values)

In [ ]:

data = {
	'stock_id': 'MU',
	'average_price': 127.32
}
update_my_portfolio(data)

### search news by keyword

In [163]:
keyword = 'ai'

news = get_technews_by_keyword(keyword)

print_news(news)


118030 : AI 智能搜尋成長逾四成！最純 AI 軟體股意藍資訊 5 月底掛牌上櫃 2025-05-06T16:12:00.000Z 

https://finance.technews.tw/2025/05/06/the-purest-ai-software/
----------

118023 : AI 聊天機器人可能被污染？四招教你快速辨識 2025-05-06T15:00:00.000Z 

https://technews.tw/2025/05/06/four-tips-to-spot-your-ai-chatbot-is-not-poisoned/
----------

118022 : Visa Pay、穩定幣卡、靈活帳戶來了！邁入 AI 驅動多元支付商務新時代 2025-05-06T14:56:00.000Z 

https://finance.technews.tw/2025/05/06/visa-accept/
----------

118021 : 蘋果 2027 年要推更大螢幕 iPhone 19 Air？ 2025-05-06T14:46:00.000Z 

https://technews.tw/2025/05/06/iphone-19-air/
----------

118010 : AI 輔助程式開發，OpenAI 傳同意 30 億美元收購新創 Windsurf 2025-05-06T13:29:00.000Z 

https://finance.technews.tw/2025/05/06/openai-reaches-agreement-to-buy-windsurf-for-usd-3-billion/
----------

117989 : Pinterest 推出 AI 視覺搜尋功能，尋找風格不再煩惱 2025-05-06T09:50:00.000Z 

https://technews.tw/2025/05/06/pinterest-updates-visual-search-with-more-ai-powered-features/
----------

117986 : 阿聯學校將推人工智慧課程，從幼兒園開始 AI 教育 2025-05-06T09:30:00.000Z 

https://tec

In [ ]:
all_news = getAllTechnews(page=1, size=100)
print_news(all_news)

#### 搜索Company news

In [ ]:
all_news = get_all_company_news(page=1, size=100)

print_news(all_news)

In [ ]:
keyword = 'ai'
search_news = get_all_company_news_by_keyword(page=1, size=100, keyword=keyword)

print_news(search_news)

#### 讀取Html 轉成json資料

In [ ]:
symbol = 'NVDA'

In [ ]:
from bs4 import BeautifulSoup
from datetime import datetime, timezone

with open('index.html', 'r', encoding='utf-8') as file:
	html_content = file.read()

soup = BeautifulSoup(html_content, 'html.parser')

news = []

for row in soup.find_all('tr', class_='cursor-pointer has-label'):
	time_cell = row.find('td', align='right')
	time_text = time_cell.get_text(strip=True).replace('Today', '')

	if 'Today' in time_text:
		time_text = time_text.replace('Today', '')
		now = datetime.now(timezone.utc)
		date_str = now.date().isoformat()
	else:
		now = datetime.now(timezone.utc)
		date_str = now.date().isoformat()

	try:
		time_obj = datetime.strptime(time_text, '%I:%M%p')
		time_24hr = time_obj.strftime('%H:%M:%S')
	except ValueError:
		# 如果时间格式无法解析，使用默认时间
		time_24hr = '00:00:00'
		
	# 创建 MySQL 支持的时间字符串
	release_time = f"{date_str} {time_24hr}"

	link = row.find('a', class_='tab-link-news')
	title = link.get_text(strip=True)
	web_url = link['href']

	publisher = row.find('span').get_text(strip=True).strip('()')

	news_item = {
        "title": title,
        "symbol": symbol,
        "release_time": release_time,
        "publisher": publisher,
        "web_url": web_url
    }

	news.append(news_item)

print(json.dumps(news[0:2], indent=4))


#### 單筆寫入

In [ ]:
from time import sleep

SLEEP_SECOND = 1 / 10
for item in news:
	res = create_one_company_news(item)
	print(res)
	sleep(SLEEP_SECOND)

#### GET statement

In [ ]:
response = get_statement_by_symbol('nvda')
    
plot_pe_forward_vs_time(response[-500:])

In [175]:
# user_subscribe_news({'newsId': 116463})
data = get_user_favorite_news()
data

{'code': 200, 'message': '成功', 'data': [{'id': 80497, 'title': '路透：OpenAI 今年將敲定晶片設計，送交台積電生產', 'release_time': '2025-02-11T01:00:00.000Z', 'publisher': '中央社', 'web_url': 'https://technews.tw/2025/02/11/openai-set-to-finalize-first-custom-chip-design-this-year/'}, {'id': 116463, 'title': 'Perplexity 挑戰 Siri，意圖在 iPhone 部署實用 AI 聊天機器人', 'release_time': '2025-04-24T07:42:00.000Z', 'publisher': '邱 倢芯', 'web_url': 'https://technews.tw/2025/04/24/perplexity-siri/'}]}


[{'id': 80497,
  'title': '路透：OpenAI 今年將敲定晶片設計，送交台積電生產',
  'release_time': '2025-02-11T01:00:00.000Z',
  'publisher': '中央社',
  'web_url': 'https://technews.tw/2025/02/11/openai-set-to-finalize-first-custom-chip-design-this-year/'},
 {'id': 116463,
  'title': 'Perplexity 挑戰 Siri，意圖在 iPhone 部署實用 AI 聊天機器人',
  'release_time': '2025-04-24T07:42:00.000Z',
  'publisher': '邱 倢芯',
  'web_url': 'https://technews.tw/2025/04/24/perplexity-siri/'}]

In [ ]:
clone_db_transactions()

In [ ]:
get_all_transactions()

In [176]:
winners = get_stock_winners()
for stock in winners:
	print(stock.get('company'),'---', stock.get('dayChg'), '%')

{'code': 200, 'message': '成功', 'data': [{'id': 46057, 'symbol': None, 'company': 'Constellation Energy', 'price': '275', 'dayChg': 10.69, 'yearChg': '37.02%', 'MCap': '86.02B', 'date': '2025-05-05T16:00:00.000Z', 'createdAt': '2025-05-07T03:00:01.000Z', 'updatedAt': '2025-05-07T03:00:01.000Z'}, {'id': 46435, 'symbol': None, 'company': 'Celanese', 'price': '49', 'dayChg': 10, 'yearChg': '-69.00%', 'MCap': '4.93B', 'date': '2025-05-05T16:00:00.000Z', 'createdAt': '2025-05-07T03:00:01.000Z', 'updatedAt': '2025-05-07T03:00:01.000Z'}, {'id': 46277, 'symbol': None, 'company': 'Leidos', 'price': '155', 'dayChg': 4.78, 'yearChg': '8.24%', 'MCap': '20.9B', 'date': '2025-05-05T16:00:00.000Z', 'createdAt': '2025-05-07T03:00:01.000Z', 'updatedAt': '2025-05-07T03:00:01.000Z'}, {'id': 46132, 'symbol': None, 'company': 'Vistra Corp', 'price': '145', 'dayChg': 3.43, 'yearChg': '77.15%', 'MCap': '49.12B', 'date': '2025-05-05T16:00:00.000Z', 'createdAt': '2025-05-07T03:00:01.000Z', 'updatedAt': '2025-05

In [177]:
losers = get_stock_losers()
for stock in losers:
	print(stock.get('company'),'---', stock.get('dayChg'), '%')

{'code': 200, 'message': '成功', 'data': [{'id': 46088, 'symbol': None, 'company': 'Regeneron Pharmaceuticals', 'price': '560', 'dayChg': -7.33, 'yearChg': '-42.29%', 'MCap': '66.28B', 'date': '2025-05-05T16:00:00.000Z', 'createdAt': '2025-05-07T03:00:01.000Z', 'updatedAt': '2025-05-07T03:00:01.000Z'}, {'id': 46307, 'symbol': None, 'company': 'Coterra Energy', 'price': '23', 'dayChg': -9.26, 'yearChg': '-18.34%', 'MCap': '18.7B', 'date': '2025-05-05T16:00:00.000Z', 'createdAt': '2025-05-07T03:00:01.000Z', 'updatedAt': '2025-05-07T03:00:01.000Z'}, {'id': 46017, 'symbol': None, 'company': 'Vertex Pharmaceuticals', 'price': '453', 'dayChg': -9.3, 'yearChg': '10.55%', 'MCap': '130.13B', 'date': '2025-05-05T16:00:00.000Z', 'createdAt': '2025-05-07T03:00:01.000Z', 'updatedAt': '2025-05-07T03:00:01.000Z'}, {'id': 45969, 'symbol': 'PLTR', 'company': 'Palantir Technologies', 'price': '109', 'dayChg': -12.05, 'yearChg': '408.69%', 'MCap': '273.66B', 'date': '2025-05-05T16:00:00.000Z', 'createdAt':